# Attention
Class = 5.

Integrate (1) Nucleus features as recored by CellProfiler Proces100
and (2) attention level recorded by the CNN.
Use Random Forest to say what CP features are predictive of attention.
Here, we do the analysis per class, just because that is easiest.

In [1]:
import datetime
import numpy as np
import pandas as pd
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split

In [2]:
ATTFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/attention_nucleus/' # Alien
ATTFILENAME='class5_avg_temp.csv'
NUCFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/Output5/'
NUCFILENAME='Process100_Nucleus.csv'

## Load labels
The labels are the temperatures of the CNN attention heatmap,
specifically an average across the bounding box of each nucleus.

In [3]:
att_df=pd.read_csv(ATTFILEPATH+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
att_df.rename(columns=cols,inplace=True)
att_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
att_df.reset_index(drop=True,inplace=True)

In [4]:
# Later, try the RandomForestRegressor. For now use bins and classification.
bins = [0, 0.3, 0.45, 0.6, 0.75, 1.0]
labels = ['to30%', 'to45%', 'to60%', 'to75%', 'to100%']
att_df['bin'] = pd.cut(att_df['AvgTemp'], bins=bins, labels=labels)
att_df

,PatchNumber,ObjectNumber,AvgTemp,bin
0,1,1,0.233538,to30%
1,1,2,0.321060,to45%
2,1,3,0.403757,to45%
3,1,4,0.457226,to60%
4,1,5,0.504176,to60%
...,...,...,...,...
53530,1590,33,0.773897,to100%
53531,1591,1,0.459364,to60%
53532,1591,2,0.264766,to30%
53533,1591,3,0.338147,to45%


In [5]:
ytrain = att_df['bin'] 
ytrain.value_counts()

to30%     11786
to100%    11588
to60%     11140
to45%     10305
to75%      8716
Name: bin, dtype: int64

## Load features
These are features of nuclei as determined by our "Process100" pipeline for CellProfiler.
These are the nuclei used to set the bounding boxes for the temperature lables.
Other than that, the features and temperatures were each derived independently.

In [6]:
nuc_df=pd.read_csv(NUCFILEPATH+NUCFILENAME)
cols={'ImageNumber':'PatchNumber'}
nuc_df.rename(columns=cols,inplace=True)
nuc_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
nuc_df.reset_index(drop=True,inplace=True)

In [7]:
Xtrain = nuc_df.drop(columns=['PatchNumber','ObjectNumber'])
Xtrain

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,...,Texture_Variance_Hematoxylin_4_02_256,Texture_Variance_Hematoxylin_4_03_256,Texture_Variance_Hematoxylin_5_00_256,Texture_Variance_Hematoxylin_5_01_256,Texture_Variance_Hematoxylin_5_02_256,Texture_Variance_Hematoxylin_5_03_256,Texture_Variance_Hematoxylin_7_00_256,Texture_Variance_Hematoxylin_7_01_256,Texture_Variance_Hematoxylin_7_02_256,Texture_Variance_Hematoxylin_7_03_256
0,257,364,194,14,168,0,179.680934,5.241245,257.0,-4.263256e-14,...,681.639591,564.722473,648.004869,678.803563,644.486105,556.292714,589.667692,861.632147,645.127433,520.479410
1,1230,1763,215,54,172,13,192.229268,31.889431,1230.0,3.126388e-13,...,457.793888,427.195252,464.462349,470.453898,463.417370,411.709306,456.646188,514.630194,486.037547,412.162634
2,582,945,239,57,212,22,225.123711,38.482818,582.0,1.705303e-13,...,242.114330,255.022783,257.044711,272.115662,240.692317,265.408639,282.593527,268.629002,258.640336,308.727980
3,833,1080,209,76,173,46,191.584634,60.704682,833.0,6.288303e-13,...,215.514820,209.908539,204.716900,216.001172,210.714399,203.749999,213.471251,230.988281,194.248272,197.194280
4,619,870,233,138,204,108,217.935380,122.276252,619.0,1.397993e-12,...,346.010112,360.099220,402.140038,309.685058,334.911450,370.105404,422.193783,263.134268,322.224580,426.499802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53530,218,275,231,300,206,289,218.536697,294.834862,218.0,-4.263256e-14,...,982.190735,1038.210680,991.631311,1097.601128,1126.843023,1190.290937,964.271943,1630.624628,1697.607939,1432.979592
53531,196,434,60,113,29,99,43.770408,104.454082,196.0,3.392842e-13,...,912.771815,1095.310204,814.818886,762.034722,961.244051,1109.953056,771.915748,711.678571,1945.555556,1326.931332
53532,406,992,122,141,90,110,104.073892,125.798030,406.0,6.679102e-13,...,1328.106904,1417.674260,1393.986496,1515.761414,1412.736777,1188.149908,1235.288744,1692.522912,1576.714266,781.383551
53533,903,1850,145,185,108,135,124.060908,161.367663,903.0,2.700062e-13,...,1155.039617,1210.211681,1189.410863,1146.025525,1160.344858,1269.514888,1229.177456,1140.045958,1187.982750,1347.723797


In [8]:
att_df=None
nuc_df=None
print('Count Nan before:',Xtrain.isna().sum().sum())
Xtrain.fillna(0,inplace=True)
print('Count Nan after:',Xtrain.isna().sum().sum())

Count Nan before: 160613
Count Nan after: 0


## Measure agreement (with a random forest classifier)
Can the RF predict the temperature of each nucleus given only the CellProfiler features?

In [9]:
print(datetime.datetime.now())
print('Cross validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('Accuracy mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-07 15:39:04.820483
Cross validation...
[0.91482208 0.91286075 0.91650322 0.91781078 0.89558233]
Accuracy mean 0.9115 +/- 0.0081


In [10]:
# TO DO: train once just to get confusion matrix

## Measure feature importance

In [11]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf1 = None
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
rf2.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf2.important_features()
pd.set_option('display.max_rows', None)
top.loc[:25]

2022-06-07 15:45:42.385420
Re-train on full training set...
2022-06-07 15:47:25.684348
...and rank the features by importance.


,0,1
0,0.067081,Location_Center_X
1,0.064245,AreaShape_Center_X
2,0.060753,Location_CenterMassIntensity_X_Hematoxylin
3,0.060525,AreaShape_BoundingBoxMaximum_X
4,0.058944,AreaShape_BoundingBoxMinimum_X
5,0.050102,Location_MaxIntensity_X_Hematoxylin
6,0.037274,Location_CenterMassIntensity_Y_Hematoxylin
7,0.035473,AreaShape_Center_Y
8,0.035132,Location_Center_Y
9,0.033754,AreaShape_BoundingBoxMinimum_Y
